# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-07-10 23:48:38.584208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-10 23:48:38.584232: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'oil'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[-1.24092248,  1.15238979,  1.34643407, ...,  3.68676672,
         0.38873035, -0.05837721],
       [-1.22552414, 11.38954577, -1.03327258, ...,  6.36218073,
         0.38776948, -0.6396642 ],
       [-1.21012579, -0.11281849,  1.2526453 , ...,  3.46046621,
         0.39545643, -0.09621246],
       ...,
       [ 1.85414469, -0.1651255 , -1.12327578, ..., -0.8513792 ,
         0.40506512, -0.68781815],
       [ 1.86954303, -0.16719707, -1.00510593, ..., -0.81220448,
         0.41083033, -0.61902679],
       [ 1.88494138, -0.16667918, -1.15230961, ..., -0.6135064 ,
         0.38008253, -0.6396642 ]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 896, 1: 41})
Ratio-> 21.9 : 1


In [10]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861
__________________________________________________
Classifier:  LGBMClassifier
[[178   2]
 [  7   1]]

TNR: 0.98889
TPR: 0.125
TNRxTPR: 0.12361
G-mean: 0.35158
__________________________________________________
Classifier:  XGBClassifier
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861
__________________________________________________
Classifier:  BaggingClassifier
[[178   2]
 [  6   2]]

TNR: 0.98889
TPR: 0.25
TNRxTPR: 0.24722
G-mean: 0.49721
__________________________________________________
Classifier:  RandomForestClassifier
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861
__________________________________________________
Ensemble predictions (majority voting):
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [11]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe217c408b0>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [12]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 716, 1: 716})
Ratio->  1 : 1.0


In [13]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[177   3]
 [  4   4]]

TNR: 0.98333
TPR: 0.5
TNRxTPR: 0.49167
G-mean: 0.70119
__________________________________________________
Classifier:  LGBMClassifier
[[178   2]
 [  5   3]]

TNR: 0.98889
TPR: 0.375
TNRxTPR: 0.37083
G-mean: 0.60896
__________________________________________________
Classifier:  XGBClassifier
[[177   3]
 [  4   4]]

TNR: 0.98333
TPR: 0.5
TNRxTPR: 0.49167
G-mean: 0.70119
__________________________________________________
Classifier:  BaggingClassifier
[[175   5]
 [  6   2]]

TNR: 0.97222
TPR: 0.25
TNRxTPR: 0.24306
G-mean: 0.49301
__________________________________________________
Classifier:  RandomForestClassifier
[[176   4]
 [  5   3]]

TNR: 0.97778
TPR: 0.375
TNRxTPR: 0.36667
G-mean: 0.60553
__________________________________________________
Ensemble predictions (majority voting):
[[177   3]
 [  4   4]]

TNR: 0.98333
TPR: 0.5
TNRxTPR: 0.49167
G-mean: 0.70119


In [14]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [15]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 716, 1: 716})
Ratio->  1 : 1.0


In [16]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[178   2]
 [  6   2]]

TNR: 0.98889
TPR: 0.25
TNRxTPR: 0.24722
G-mean: 0.49721
__________________________________________________
Classifier:  LGBMClassifier
[[179   1]
 [  7   1]]

TNR: 0.99444
TPR: 0.125
TNRxTPR: 0.12431
G-mean: 0.35257
__________________________________________________
Classifier:  XGBClassifier
[[178   2]
 [  6   2]]

TNR: 0.98889
TPR: 0.25
TNRxTPR: 0.24722
G-mean: 0.49721
__________________________________________________
Classifier:  BaggingClassifier
[[175   5]
 [  6   2]]

TNR: 0.97222
TPR: 0.25
TNRxTPR: 0.24306
G-mean: 0.49301
__________________________________________________
Classifier:  RandomForestClassifier
[[177   3]
 [  6   2]]

TNR: 0.98333
TPR: 0.25
TNRxTPR: 0.24583
G-mean: 0.49582
__________________________________________________
Ensemble predictions (majority voting):
[[179   1]
 [  7   1]]

TNR: 0.99444
TPR: 0.125
TNRxTPR: 0.12431
G-mean: 0.35257


In [17]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-07-10 23:49:32.579279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-10 23:49:32.579303: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-10 23:49:32.579320: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-07-10 23:49:32.579527: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 716, 1.0: 716})
Ratio->  1 : 1.0


In [19]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861
__________________________________________________
Classifier:  LGBMClassifier
[[179   1]
 [  7   1]]

TNR: 0.99444
TPR: 0.125
TNRxTPR: 0.12431
G-mean: 0.35257
__________________________________________________
Classifier:  XGBClassifier
[[177   3]
 [  6   2]]

TNR: 0.98333
TPR: 0.25
TNRxTPR: 0.24583
G-mean: 0.49582
__________________________________________________
Classifier:  BaggingClassifier
[[178   2]
 [  7   1]]

TNR: 0.98889
TPR: 0.125
TNRxTPR: 0.12361
G-mean: 0.35158
__________________________________________________
Classifier:  RandomForestClassifier
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861
__________________________________________________
Ensemble predictions (majority voting):
[[179   1]
 [  6   2]]

TNR: 0.99444
TPR: 0.25
TNRxTPR: 0.24861
G-mean: 0.49861


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [20]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe270ad0790>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [25]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.1)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 716, 0: 72})
Ratio->  1 : 9.9


In [26]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[145  35]
 [  1   7]]

TNR: 0.80556
TPR: 0.875
TNRxTPR: 0.70486
G-mean: 0.83956
__________________________________________________
Classifier:  LGBMClassifier
[[145  35]
 [  2   6]]

TNR: 0.80556
TPR: 0.75
TNRxTPR: 0.60417
G-mean: 0.77728
__________________________________________________
Classifier:  XGBClassifier
[[146  34]
 [  2   6]]

TNR: 0.81111
TPR: 0.75
TNRxTPR: 0.60833
G-mean: 0.77996
__________________________________________________
Classifier:  BaggingClassifier
[[145  35]
 [  1   7]]

TNR: 0.80556
TPR: 0.875
TNRxTPR: 0.70486
G-mean: 0.83956
__________________________________________________
Classifier:  RandomForestClassifier
[[155  25]
 [  2   6]]

TNR: 0.86111
TPR: 0.75
TNRxTPR: 0.64583
G-mean: 0.80364
__________________________________________________
Ensemble predictions (majority voting):
[[149  31]
 [  1   7]]

TNR: 0.82778
TPR: 0.875
TNRxTPR: 0.72431
G-mean: 0.85106


In [27]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [32]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.1)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 716, 0: 72})
Ratio->  1 : 9.9


In [33]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[152  28]
 [  1   7]]

TNR: 0.84444
TPR: 0.875
TNRxTPR: 0.73889
G-mean: 0.85959
__________________________________________________
Classifier:  LGBMClassifier
[[152  28]
 [  2   6]]

TNR: 0.84444
TPR: 0.75
TNRxTPR: 0.63333
G-mean: 0.79582
__________________________________________________
Classifier:  XGBClassifier
[[147  33]
 [  2   6]]

TNR: 0.81667
TPR: 0.75
TNRxTPR: 0.6125
G-mean: 0.78262
__________________________________________________
Classifier:  BaggingClassifier
[[159  21]
 [  3   5]]

TNR: 0.88333
TPR: 0.625
TNRxTPR: 0.55208
G-mean: 0.74302
__________________________________________________
Classifier:  RandomForestClassifier
[[157  23]
 [  2   6]]

TNR: 0.87222
TPR: 0.75
TNRxTPR: 0.65417
G-mean: 0.80881
__________________________________________________
Ensemble predictions (majority voting):
[[154  26]
 [  2   6]]

TNR: 0.85556
TPR: 0.75
TNRxTPR: 0.64167
G-mean: 0.80104


In [61]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=1.5)

In [62]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.06)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 1057, 0.0: 43})
Ratio->  1 : 24.6


In [63]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[140  40]
 [  2   6]]

TNR: 0.77778
TPR: 0.75
TNRxTPR: 0.58333
G-mean: 0.76376
__________________________________________________
Classifier:  LGBMClassifier
[[147  33]
 [  2   6]]

TNR: 0.81667
TPR: 0.75
TNRxTPR: 0.6125
G-mean: 0.78262
__________________________________________________
Classifier:  XGBClassifier
[[148  32]
 [  2   6]]

TNR: 0.82222
TPR: 0.75
TNRxTPR: 0.61667
G-mean: 0.78528
__________________________________________________
Classifier:  BaggingClassifier
[[143  37]
 [  1   7]]

TNR: 0.79444
TPR: 0.875
TNRxTPR: 0.69514
G-mean: 0.83375
__________________________________________________
Classifier:  RandomForestClassifier
[[144  36]
 [  2   6]]

TNR: 0.8
TPR: 0.75
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Ensemble predictions (majority voting):
[[143  37]
 [  2   6]]

TNR: 0.79444
TPR: 0.75
TNRxTPR: 0.59583
G-mean: 0.7719


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
